In [10]:
import os
import pandas as pd

os. chdir("c:\\Users\\simon\\OneDrive\\Dokumente\\UNILU\\3 - HS24\\1 MA\\business-reports-nlp")

df = pd.read_csv('texts.csv')
df = df.sample(frac = 1)
df

,file_name,file_path,source,content
76,mckinsey.Global payments 2017_ Amid rapid chan...,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,! Financial Services Report October 2017 G...
44,deloitte.mobile-money-payment-industry-marketi...,reports_digipay_txt/consultancies_pdfminer_cle...,deloitte,Mobile money A payment industry revolution im...
57,global banking annual review 2022 banking on a...,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,Banking on a sustainable path. Global Bankin...
1,accenture.PSD2-Seizing-Opportunities-EU-Paymen...,reports_digipay_txt/consultancies_pdfminer_cle...,accenture,Accenture Payment Services Seizing the Opport...
15,capgemini.world_payments_report.2008.txt,reports_digipay_txt/consultancies_pdfminer_cle...,capgemini,World Payments Report 2008 Mobile Payments M...
...,...,...,...,...
90,mckinsey.Monetizing data- A new source of valu...,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,! Financial Services Article September 2017...
87,mckinsey.MENA entrepreneurship article-final v...,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,Entrepreneurship in the Middle East and No...
35,world-payments-report-wpr-2020_web.txt,reports_digipay_txt/consultancies_pdfminer_cle...,capgemini,WORLD PAYMENTSREPORT2020 TABLE OF CONTENTS ...
42,Deloitte.banerjee.winnington.Smart-device-smar...,reports_digipay_txt/consultancies_pdfminer_cle...,deloitte,Deloitte’s mPayments practice brings together ...


In [25]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=10, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=2,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english'))# + ['http', 'https', 'amp', 'com']

# we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

In [27]:
from bertopic import BERTopic

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(df['content'])

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2023-09-20 23:13:44,928 - BERTopic - Transformed documents to Embeddings
2023-09-20 23:13:50,086 - BERTopic - Reduced dimensionality
2023-09-20 23:13:50,139 - BERTopic - Clustered reduced embeddings


In [28]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,113,0_payments_banks_payment_services,"[payments, banks, payment, services, digital]",[WORLD PAYMENTS REPORT 2017 TABLE OF CONTE...
1,1,11,1_global_percent_mckinsey_digital,"[global, percent, mckinsey, digital, skills]",[T h e f u t u r e o f w o r k ...


In [29]:
model.visualize_barchart()